In [2]:
# Python code written by Wei-Chen CHU, ICOB imaging core, Academia Sinica
# Last updated: 2024/9/26 (code), 2024/10/2(comment)
# This code aims to process a time series of TIRF images to find Glut10_Rab5_Colocalize vesicles for further vesicle tracking analysis by Trackmate.
# Test enviroment: devbio-napari (https://github.com/haesleinhuepf/devbio-napari)

import numpy as np
import pyclesperanto_prototype as cle
import napari
import matplotlib.pyplot as plt
from skimage.io import imread
import napari_simpleitk_image_processing as nsitk  # version 0.4.5

cle.select_device('RTX 3050') # You can check the GPU device name using cle.available_device_names()
viewer = napari.Viewer()

# load the image data, you need specify the paths of your images
img_Glu = imread(r"D:\Anu_TIRF\TIRF_20220726_ANU\CELL2\TF488_0hr_2.tif") # Glut-10-GFP image
img_Rab5 = imread(r"D:\Anu_TIRF\TIRF_20220726_ANU\CELL2\TF532_0hr_2.tif") # Rab5-RFP image

# The time-series cell_label was generated by TrackMate-Cellpose to identify regions of interest for colocalization analysis.
# We applied a median filter with a radius of 4 to the Rab-5 images for use with TrackMate-Cellpose (Cyto3 model).
cell_label = imread(r"D:\Anu_TIRF\TIRF_20220726_ANU\CELL2\TF532_0hr_2_Label.tif") 

viewer.add_image(img_Glu, name='Glut10_RAW', colormap='green', blending='additive')
viewer.add_image(img_Rab5, name='Rab5_RAW', colormap='magenta', blending='additive')

#DoG Parameter (apply to both channels)
DoG_Glu_small = 1
DoG_Glu_large = 4

DoG_Rab_small = 1
DoG_Rab_large = 4

#Overlap_score thresholding parameter
min_overlap_score = 0.5
max_overlap_score = 1.0

In [ ]:
# Image processing workflow:
# Glut-10 image -> Difference of Gaussian filter -> Voronoi_Otsu Labeling -> Glut-10 labels
# Rab5 image -> Difference of Gaussian filter -> threshold_triangle -> Rab5 mask

img_glu_DoG_filtered = np.zeros_like(img_Glu, dtype='uint32')
img_rab_DoG_filtered = np.zeros_like(img_Glu, dtype='uint32')
glut10_label = np.zeros_like(img_Glu, dtype='uint32')
rab5_mask = np.zeros_like(img_Rab5, dtype='uint32')
overlap_score_maps = np.zeros_like(img_Glu, dtype='float32')
colocalize_labels = np.zeros_like(img_Glu, dtype='uint32')
non_colocalize_labels = np.zeros_like(img_Glu, dtype='uint32')
mid_colocalize_labels = np.zeros_like(img_Glu, dtype='uint32')

# create lists for store the colocalization data
colocalize_ratio_list = []
colocalization_cell_number = []
non_colocalize_cell_number = []
mid_colocalize_cell_number = []

for t in range(img_Glu.shape[0]):
    # Process Glut10 singal -> Voroni-Otsu labeling -> Glut10 labels
    gpu_input = img_Glu[t,:,:]
    gpu_DoG = cle.difference_of_gaussian(gpu_input, sigma1_x=DoG_Glu_small, sigma1_y=DoG_Glu_small, sigma2_x=DoG_Glu_large, sigma2_y=DoG_Glu_large)   
    gpu_DoG_positive = cle.pull(gpu_DoG)
    gpu_DoG_positive[gpu_DoG_positive <0] =0    
    gpu_DoG_positive_crop = gpu_DoG_positive * cell_label[t,:,:]  # Apply the cell_label as "Region of Interest"
    gpu_Glu_label = cle.voronoi_otsu_labeling(gpu_DoG_positive_crop, spot_sigma=1, outline_sigma=1)

    # Store the single time point data into a time-seies numpy array
    img_glu_DoG_filtered[t] = gpu_DoG_positive_crop
    glut10_label[t] = cle.pull(gpu_Glu_label)

    #Process Rab5 signal -> threshold_triangle -> Rab5 Mask
    gpu_input = img_Rab5[t,:,:]
    gpu_DoG = cle.difference_of_gaussian(gpu_input, sigma1_x=DoG_Rab_small, sigma1_y=DoG_Rab_small, sigma2_x=DoG_Rab_large, sigma2_y=DoG_Rab_large)
    gpu_DoG_positive = cle.pull(gpu_DoG)
    gpu_DoG_positive[gpu_DoG_positive <0] = 0
    gpu_DoG_positive_crop = gpu_DoG_positive * cell_label[t,:,:]
    img_rab_DoG_filtered[t] = gpu_DoG_positive_crop  
    gpu_mask = nsitk.threshold_triangle(gpu_DoG_positive_crop)

    # Store the single time point data into a time-seies numpy array
    rab5_mask[t] = cle.pull(gpu_mask)
   
    # Overlap analysis, remove the low overlap_score labels
    gpu_overlap_score_map = cle.mean_intensity_map(gpu_mask, gpu_Glu_label) # gpu_mask here is the Rab5 mask
    overlap_score_maps[t] = cle.pull(gpu_overlap_score_map)
    gpu_colocalize_label = cle.exclude_labels_with_map_values_out_of_range(gpu_overlap_score_map, gpu_Glu_label, minimum_value_range=min_overlap_score , maximum_value_range=max_overlap_score)
    gpu_non_colocalize_label = cle.exclude_labels_with_map_values_out_of_range(gpu_overlap_score_map, gpu_Glu_label, minimum_value_range=0 , maximum_value_range=0)
    gpu_mid_colocalize_label = cle.exclude_labels_with_map_values_out_of_range(gpu_overlap_score_map, gpu_Glu_label, minimum_value_range=0.01 , maximum_value_range=min_overlap_score)
        
    colocalize_labels[t] = cle.pull(gpu_colocalize_label)
    non_colocalize_labels[t] = cle.pull(gpu_non_colocalize_label)
    mid_colocalize_labels[t] = cle.pull(gpu_mid_colocalize_label)

    colocalization_cell_number.append(np.max(np.unique(colocalize_labels[t])))
    mid_colocalize_cell_number.append(np.max(np.unique(mid_colocalize_labels[t])))
    non_colocalize_cell_number.append(np.max(np.unique(non_colocalize_labels[t])))
  
    #Count the number of Glut10 labels and glut10-Rab5 labels
    num_glut10_labels = len(np.unique(glut10_label[t])) - 1
    num_glut10_rab5_labels = len(np.unique(colocalize_labels[t])) -1

    #Count the colocalize ratio
    colocalize_ratio = num_glut10_rab5_labels/num_glut10_labels
    colocalize_ratio_list.append(colocalize_ratio)

# show the images in the napari viewer, you can save the necessary image files for further tracking analysis
viewer.add_image(img_glu_DoG_filtered, name='Glut10_DoG_Filtered', colormap='green', blending='additive')
viewer.add_image(img_rab_DoG_filtered, name='Rab5_DoG_Filtered', colormap='magenta', blending='additive')
# viewer.add_labels(cell_mask, name= 'Cell_mask', blending= 'additive')
viewer.add_labels(glut10_label, name='Glut10_label', blending='additive')
viewer.add_labels(rab5_mask, name='Rab5_mask', blending='additive')
viewer.add_image(overlap_score_maps, colormap='gray',blending='additive')
viewer.add_labels(colocalize_labels, name ='Glut10_Rab5_Colocalize', blending='additive')
viewer.add_labels(non_colocalize_labels)
viewer.add_labels(mid_colocalize_labels)
# viewer.add_image(glut10_intensites_maps, colormap='inferno', blending='additive')

In [ ]:
# This part of the code convert the colocalization data into a excel file for further analysis

import pandas as pd
import matplotlib.pyplot as plt

# Create a dictionary with your data
data = {
  'Time': range(len(colocalization_cell_number)),
  'Colocalized': colocalization_cell_number,
  'Mid-colocalized': mid_colocalize_cell_number,
  'Non-colocalized': non_colocalize_cell_number
}
# Create a DataFrame

df = pd.DataFrame(data)

# Export to Excel
excel_filename = 'colocalization_data.xlsx'
df.to_excel(excel_filename, index=False)
print(f"Data exported to {excel_filename}")

In [ ]:
# Create the plot for the label number changes of three different type of glut-10 labels (colocalized, mid-colocalized, non-colocalized)
plt.figure(figsize=(10, 6))
plt.plot(df['Time'], df['Colocalized'], label='Colocalized', marker='o')
plt.plot(df['Time'], df['Mid-colocalized'], label='Mid-colocalized', marker='s')
plt.plot(df['Time'], df['Non-colocalized'], label='Non-colocalized', marker='^')

plt.xlabel('Time')
plt.ylabel('Number of Cells')
plt.title('Colocalization Over Time')
plt.legend()
plt.grid(True)

# Save the plot
plot_filename = 'colocalization_plot.png'
plt.savefig(plot_filename)
print(f"Plot saved as {plot_filename}")

# Display the plot
plt.show()

In [ ]:
# plot the Glut10-Rab5/Colocalize Ratio
plt.xlim(0, 240)
plt.ylim(0, 1)

# Define label interval
interval = 30  # Replace with your desired interval
plt.xticks(range(0, len(colocalize_ratio_list), interval))


plt.plot(colocalize_ratio_list)
plt.ylim(0, 1)
plt.title(f'Glut10-Rab5/Colocalize Ratio (min_overlap_score={min_overlap_score})',)
plt.xlabel('Time frame')
plt.ylabel('colocalize_ratio')

In [ ]:
# This code cell used to process the tracking data generated by the Trackmate
# If the tracking label images contain value with '-1', you can apply this code cell to remove the '-1' label from the tracking images

# load the image data, you need specify the paths of your images
tracking_label = cle.imread(r'G:\ICOB_YCLee\20240529\1hr\20240627\LblImg_1hr_colocalize_labels.tif')
tracking_label_clean = np.zeros_like(tracking_label, dtype='int32')

for t in range(tracking_label.shape[0]):
    gpu_input = tracking_label[t,:,:]
    gpu_tracking_label_clean = cle.exclude_labels_with_map_values_equal_to_constant(gpu_input, gpu_input, constant = -1)
    tracking_label_clean[t] = gpu_tracking_label_clean

viewer.add_labels(tracking_label_clean)